# Generate Dyanmical Regression Matrix


Import Libraries


In [1]:
import sympy as p
import numpy as np
import sys

# SYMBOLIC.py is in parent directory
sys.path.append("../..")
from SYMBOLIC import *

## 1. Kinematics with PoE


### 1.1 Variable Definitions


Define time $t$ and angles.


In [2]:
t = p.Symbol("t")

# Differentialable Variables
theta_1_t = p.Function("theta_1")(t)
theta_2_t = p.Function("theta_2")(t)

# Undifferentialable Variables
theta_1, theta_2 = p.symbols("theta_1 theta_2")
v_1, v_2 = p.symbols("v_1 v_2")
a_1, a_2 = p.symbols("a_1 a_2")

Define robot parameters.


In [3]:
m1, m2, l1, l2, lc1, lc2, Jc1, Jc2, g = p.symbols("m1 m2 l1 l2 lc1 lc2 Jc1 Jc2 g")

# Jiang's Simulation
l1 = 1.7
l2 = 2.1
g = 9.8

Define substitution functions from differentialable variables to undifferentialable variables.

In [4]:
def ThetaT2Theta(Expression):
    return (
        Expression.subs(theta_1_t.diff(t, 2), a_1)
        .subs(theta_2_t.diff(t, 2), a_2)
        .subs(theta_1_t.diff(t), v_1)
        .subs(theta_2_t.diff(t), v_2)
        .subs(theta_1_t, theta_1)
        .subs(theta_2_t, theta_2)
    )

Define geometric parameters. We apply **Modified D-H** notation. Noting that $\square_{i}$ refers to $\square$ of joint $i$, $\square_{ci}$ refers to $\square$ of rod $i$.


In [5]:
# Joint DH Parameters
x0, x1, x2 = p.symbols("x_0 x_1 x_2")
y0, y1, y2 = p.symbols("y_0 y_1 y_2")
z0, z1, z2 = p.symbols("z_0 z_1 z_2")

# Link CoM Parameters
xc1, xc2 = p.symbols("x_c1 x_c2")
yc1, yc2 = p.symbols("y_c1 y_c2")
zc1, zc2 = p.symbols("z_c1 z_c2")

From the configuration, we got

In [6]:
# Joint Parameters
x0, y0, z0 = 0, 0, 0
x1, y1, z1 = 0, 0, 0
x2, y2, z2 = l1, 0, 0

# CoM Parameters
xc1, yc1, zc1 = lc1, 0, 0
xc2, yc2, zc2 = lc2, 0, 0

# RPY Angles
RPY_0_1 = [0, 0, 0]
RPY_1_2 = [0, 0, 0]
RPY_1_c1 = [0, 0, 0]
RPY_2_c2 = [0, 0, 0]

### 1.2 Forward Kinematics


Transformation matrixes at $\vec{0}$. `T_i_j_0` refers to $T_{i}^{j} (\vec{0}) $.


In [7]:
T_1_0_0 = Translation_4x4(p.Matrix([x1, y1, z1])) @ Rotation_RPY_4x4(RPY_0_1)
T_2_1_0 = Translation_4x4(p.Matrix([x2, y2, z2])) @ Rotation_RPY_4x4(RPY_1_2)

Joint positions at $\vec{0}$.

In [8]:
def Position(Transformation_i_j, P_i=p.Matrix([0, 0, 0, 1])):
    return (Transformation_i_j @ P_i)[:3, :]


T_1_0_0 = T_1_0_0
T_2_0_0 = T_1_0_0 @ T_2_1_0

r_1_0 = Position(T_1_0_0)
r_2_0 = Position(T_2_0_0)

Joint poses at $\vec{0}$.

In [9]:
omega_1_0 = p.Matrix([0, 0, 1]).reshape(3, 1)
omega_2_0 = Rotation_RPY_4x4(RPY_1_2)[:3, :3] @ omega_1_0

Joint unitary twists at $\vec{0}$.

In [10]:
xi_1_0 = Joint2Twist(omega_1_0, r_1_0)
xi_2_0 = Joint2Twist(omega_2_0, r_2_0)

Transformation matrixes at $\vec{\theta}$ computed with PoE.


In [11]:
T_1_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ T_1_0_0
T_2_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ MatrixExp_4x4(xi_2_0, theta_2_t) @ T_2_0_0

Joint positions and poses at $\vec{\theta}$.

In [12]:
# Positions
r_1 = Position(T_1_0)
r_2 = Position(T_2_0)

# Poses
omega_1 = p.simplify(MatrixExp_3x3(omega_1_0, theta_1_t) @ omega_1_0)
omega_2 = p.simplify(
    MatrixExp_3x3(omega_1_0, theta_1_t)
    @ MatrixExp_3x3(omega_2_0, theta_2_t)
    @ omega_2_0
)

Spatial velocities at $\vec{\theta}$.


In [13]:
SpatialVelocity_Rot_1 = p.simplify(omega_1 * theta_1_t.diff(t))
SpatialVelocity_Rot_2 = p.simplify(
    omega_1 * theta_1_t.diff(t) + omega_2 * theta_2_t.diff(t)
)
SpatialVelocity_Trans_1 = p.simplify(p.simplify(r_1).diff(t))
SpatialVelocity_Trans_2 = p.simplify(p.simplify(r_2).diff(t))

SpatialVelocity_1 = SpatialVelocity_Rot_1.col_join(SpatialVelocity_Trans_1)
SpatialVelocity_2 = SpatialVelocity_Rot_2.col_join(SpatialVelocity_Trans_2)

## 2. Dynamical Regression Matrix


Dynamical parameter vectors $\vec{p}_{i}$.


In [14]:
p_1 = p.Matrix(
    [
        Jc1 / 2 + m1 * (yc1**2 + zc1**2),
        -(0 + m1 * xc1 * yc1),
        -(0 + m1 * xc1 * zc1),
        Jc1 / 2 + m1 * (xc1**2 + zc1**2),
        -(0 + m1 * yc1 * zc1),
        Jc1 + m1 * (xc1**2 + yc1**2),
        m1 * xc1,
        m1 * yc1,
        m1 * zc1,
        m1,
    ]
).reshape(10, 1)
p_2 = p.Matrix(
    [
        Jc2 / 2 + m2 * (yc2**2 + zc2**2),
        -(0 + m2 * xc2 * yc2),
        -(0 + m2 * xc2 * zc2),
        Jc2 / 2 + m2 * (xc2**2 + zc2**2),
        -(0 + m2 * yc2 * zc2),
        Jc2 + m2 * (xc2**2 + yc2**2),
        m2 * xc2,
        m2 * yc2,
        m2 * zc2,
        m2,
    ]
).reshape(10, 1)

$\tilde{T}^{i}$ s.


In [15]:
Ttilde_1 = p.simplify(Ttilde_10x1(SpatialVelocity_1, T_1_0))
Ttilde_2 = p.simplify(Ttilde_10x1(SpatialVelocity_2, T_2_0))

Verify $\tilde{T}^{i}$ s.


In [16]:
p.simplify(ThetaT2Theta(Ttilde_2.T @ p_2))

Matrix([[1.7*lc2*m2*v_1*(v_1 + v_2)*cos(theta_2) + 1.445*m2*v_1**2 + (Jc2 + lc2**2*m2)*(v_1 + v_2)**2/2]])

$\tilde{V}^{i}$ s.


In [17]:
Vtilde_1 = p.simplify(Vtilde_10x1(r_1, p.Matrix([0, -g, 0]).reshape(3, 1), T_1_0))
Vtilde_2 = p.simplify(Vtilde_10x1(r_2, p.Matrix([0, -g, 0]).reshape(3, 1), T_2_0))

Verify $\tilde{V}^{i}$ s.


In [18]:
p.simplify(ThetaT2Theta(Vtilde_2.T @ p_2))

Matrix([[m2*(9.8*lc2*sin(theta_1 + theta_2) + 16.66*sin(theta_1))]])

$\tilde{T}$ and $\tilde{V}$.


In [19]:
Ttilde = Ttilde_1.col_join(Ttilde_2)
Vtilde = Vtilde_1.col_join(Vtilde_2)

Define $\vec{\theta}$ and $\vec{\dot{\theta}}$.


In [20]:
theta = p.Matrix([theta_1_t, theta_2_t])
theta_dot = p.Matrix([theta_1_t.diff(t), theta_2_t.diff(t)])

Dynamical regression matrix $Y_{\text{Full}}$.


In [21]:
Y_Full = p.simplify(
    ThetaT2Theta(
        VecDiff(Ttilde, theta_dot).diff(t)
        - VecDiff(Ttilde, theta)
        + VecDiff(Vtilde, theta)
    )
)
Y_Full

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,       a_1,         0,                                                                                                              9.8*cos(theta_1),                                                                           0,                                                                                                              -9.8*sin(theta_1),                                                                            0, 0, 0,                             0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, a_1 + a_2, a_1 + a_2, 3.4*a_1*cos(theta_2) + 1.7*a_2*cos(theta_2) - 3.4*v_1*v_2*sin(theta_2) - 1.7*v_2**2*sin(theta_2) + 9.8*cos(theta_1 + theta_2), 1.7*a_1*cos(theta_2) + 1.7*v_1**2*sin(theta_2) + 9.8*cos(theta_1 + theta_2), -3.4*a_1*sin(theta_2) - 1.7*a_2*sin(theta_2) - 3.4*v_1*v_2*cos(theta_2) - 1.7*v_2**2*cos(theta_2) - 9.8*sin(theta_1 + theta_2), -1.7*a_1*sin(theta_2) + 1.7*v_1**2*cos(theta_2) - 9.8*sin(theta_1 + theta_2), 0, 0, 2.89*a_1 + 16.66*c

Remove zero columns in $Y_{\text{Full}}$ to get $Y$.


In [32]:
cols = [
    i for i in range(Y_Full.shape[1]) if Y_Full[:, i] != p.zeros(Y_Full.shape[0], 1)
]
Y = Y_Full[:, cols]
print("Y's shape is", Y.shape)
Y

Y's shape is (2, 7)


Matrix([
[      a_1,         0,                                                                                                              9.8*cos(theta_1),                                                                           0,                                                                                                              -9.8*sin(theta_1),                                                                            0,                             0],
[a_1 + a_2, a_1 + a_2, 3.4*a_1*cos(theta_2) + 1.7*a_2*cos(theta_2) - 3.4*v_1*v_2*sin(theta_2) - 1.7*v_2**2*sin(theta_2) + 9.8*cos(theta_1 + theta_2), 1.7*a_1*cos(theta_2) + 1.7*v_1**2*sin(theta_2) + 9.8*cos(theta_1 + theta_2), -3.4*a_1*sin(theta_2) - 1.7*a_2*sin(theta_2) - 3.4*v_1*v_2*cos(theta_2) - 1.7*v_2**2*cos(theta_2) - 9.8*sin(theta_1 + theta_2), -1.7*a_1*sin(theta_2) + 1.7*v_1**2*cos(theta_2) - 9.8*sin(theta_1 + theta_2), 2.89*a_1 + 16.66*cos(theta_1)]])

Dynamical regression vector $\vec{y}$.


In [23]:
y = Y.reshape(Y.shape[0] * Y.shape[1], 1)
y

Matrix([
[                                                                                                                           a_1],
[                                                                                                                             0],
[                                                                                                              9.8*cos(theta_1)],
[                                                                                                                             0],
[                                                                                                             -9.8*sin(theta_1)],
[                                                                                                                             0],
[                                                                                                                             0],
[                                                                                

Generate MATLAB function.


In [24]:
import os

In [25]:
CreateMatlabFunction(
    os.path.dirname(os.getcwd()) + "\\utils\\DynRegVec",
    "DynRegVec",
    y,
    [theta_1, theta_2, v_1, v_2, a_1, a_2],
)